In [1]:
import csv
import numpy as np
import math
import pandas as pd

In [2]:
file = open("diabetes.csv", "r")
dataset = csv.reader(file)#đọc dữ liệu đầu vào
dataset = np.array(list(dataset))#chuyển về dạng ma trận
dataset = np.delete(dataset,0,0)#xóa header
file.close()
#Phân chia tập dữ liệu về tập Test và tập Training
dataset = dataset.astype(float)
trainingData = dataset[:538]#Tập training bao gồm 500 bản ghi dữ liệu đầu tiên
testData = dataset[538:]#Tập test bao gồm các bản ghi dữ liệu còn lại

In [3]:
print(pd.DataFrame(trainingData,columns=["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age","Outcome"]))

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6.0    148.0           72.0           35.0      0.0  33.6   
1            1.0     85.0           66.0           29.0      0.0  26.6   
2            8.0    183.0           64.0            0.0      0.0  23.3   
3            1.0     89.0           66.0           23.0     94.0  28.1   
4            0.0    137.0           40.0           35.0    168.0  43.1   
..           ...      ...            ...            ...      ...   ...   
533          6.0     91.0            0.0            0.0      0.0  29.8   
534          1.0     77.0           56.0           30.0     56.0  33.3   
535          4.0    132.0            0.0            0.0      0.0  32.9   
536          0.0    105.0           90.0            0.0      0.0  29.6   
537          0.0     57.0           60.0            0.0      0.0  21.7   

     DiabetesPedigreeFunction   Age  Outcome  
0                       0.627  50.0      1.0  
1                

In [4]:
print(pd.DataFrame(testData,columns=["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age","Outcome"]))

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            0.0    127.0           80.0           37.0    210.0  36.3   
1            3.0    129.0           92.0           49.0    155.0  36.4   
2            8.0    100.0           74.0           40.0    215.0  39.4   
3            3.0    128.0           72.0           25.0    190.0  32.4   
4           10.0     90.0           85.0           32.0      0.0  34.9   
..           ...      ...            ...            ...      ...   ...   
225         10.0    101.0           76.0           48.0    180.0  32.9   
226          2.0    122.0           70.0           27.0      0.0  36.8   
227          5.0    121.0           72.0           23.0    112.0  26.2   
228          1.0    126.0           60.0            0.0      0.0  30.1   
229          1.0     93.0           70.0           31.0      0.0  30.4   

     DiabetesPedigreeFunction   Age  Outcome  
0                       0.804  23.0      0.0  
1                

In [5]:
def computeDistance(data1,data2):
  res = 0;
  for i in range(8):
    res += ( float(data1[i]) - float(data2[i])  )**2
  return math.sqrt(res)


In [6]:
def computeKNN(trainingData,point,k):
  distance = []#mảng chứa khoảng cách các điểm
  for data in trainingData:
    #khai bao dictionary bao gồm nhãn và khoảng cách từ các điểm trong tập training đến điểm được xét
    label_distance = {
        "label" : data[-1],
        "dis": computeDistance(data,point)
    }
    distance.append(label_distance)
    #Sắp xếp lại mảng theo thứ tự tăng dần dựa trên khoảng cách
  distance.sort(key = lambda x: x["dis"])
  label = []
    #Lấy ra label của k điểm gần nhất đơi với điểm được xét
  for element in distance:
    label.append(element["label"])
  return label[:k]


In [7]:
def predict(label):
  num1 = 0
  num0 = 0
#Đếm số lượng mỗi nhãn trong k điểm gần nhất
  for element in label:
    if element == 1:
      num1 = num1 + 1
    else:
      num0 = num0 + 1
#so sánh số lượng mỗi nhãn để chọn ra nhãn gán cho điểm
  if num1 > num0 :
    return 1
  else:
    return 0

In [8]:
def evaluate(x,y):
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    if (x==1 and y == 1):
        tp = 1
    elif (x == 1 and y == 0):
        fn = 1
    elif (y == 1):
        fp = 1
    else:
        tn = 1
    return [tp,fn,fp,tn]

In [9]:
k = 5
TP = 0
FN = 0  
FP = 0  
TN = 0  
#Thực hiện phân loại dữ liệu trong TestData
for element in testData:
    label = computeKNN(trainingData, element, k)
    #print("True = ", element[-1], "Predict = ", predict(label))
    tp, fn, fp, tn = evaluate(int(element[-1]), predict(label))  
    TP += tp
    FN += fn
    FP += fp
    TN += tn
data_size = len(testData)
#print(data_size)
#Hiển thi số lượng các nhãn dán của 4 nhóm đánh giá chất lượng
print("TP =", TP) 
print("FN =", FN) 
print("FP =", FP) 
print("TN =", TN) 
#Tính toán các chỉ số để đánh giá độ chính xác của thuật toán
accuracy =round(float( (TP + TN)/data_size),2)
precision = float(TP/(TP+FP)) # tính tỉ lệ số người thực sự bị bệnh trong những người được dự đoán bị bệnh
recall = float(TP/(TP+FN)) #tính tỉ lệ dự đoán chính xác trong những người bị bệnh

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")

TP = 47
FN = 32
FP = 31
TN = 120
Accuracy: 73.00%
Precision: 60.26%
Recall: 59.49%


# Tính KNN dựa trên trọng số

def new_computeKNN(trainingData,point,k):
  distance = []
  for data in trainingData:
    #khai bao dictionary
    label_distance = {
        "label" : data[-1],
        "dis": computeDistance(data,point)
    }
    distance.append(label_distance)
  distance.sort(key = lambda x: x["dis"])

  return distance[:k]

In [11]:
def new_predict(label):
  weight1 = 0
  weight2 = 0
  for element in label:
    if int(element["label"]) == 1:
      weight1 += 1/float(element["dis"])
    else:
      weight2 += 1/float(element["dis"])
  if weight1 > weight2: 
    return 1
  else:
    return 0

In [15]:
data_size = len(testData)
print(768-data_size)

538


In [24]:
k = 23
TP = 0
FN = 0
FP = 0
TN = 0

for element in testData:
    label = new_computeKNN(trainingData, element, k)
    #print("True = ", element[-1], "Predict = ", predict(label))

    tp, fn, fp, tn = evaluate(int(element[-1]), new_predict(label))
    TP += tp
    FN += fn
    FP += fp
    TN += tn
#print(data_size)
print("TP =", TP) # True Positive: người bị bệnh được dự đoán là bị bệnh
print("FN =", FN) # False Negative: người bị bệnh được dự đoán là không có bệnh
print("FP =", FP) # False Positive: người không bị bệnh được dự đoán là bị bệnh
print("TN =", TN) # True Negative: người không bị bệnh được dự đoán là không bị bệnh

accuracy =round(float( (TP + TN)/data_size),2)
precision = float(TP/(TP+FP)) # tính tỉ lệ số người thực sự bị bệnh trong những người được dự đoán bị bệnh
recall = float(TP/(TP+FN)) #tính tỉ lệ dự đoán chính xác trong những người bị bệnh

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")

TP = 41
FN = 38
FP = 16
TN = 135
Accuracy: 77.00%
Precision: 71.93%
Recall: 51.90%


# Lọc dữ liệu vô lý

In [25]:
numeric_dataset = dataset.astype(float)

# Tạo một mảng boolean với điều kiện
condition = np.all(numeric_dataset[:, 1:8] != 0, axis=1)

# Lọc dataset ban đầu dựa trên mảng boolean
filtered_dataset = numeric_dataset[condition]

print(filtered_dataset)

[[  1.     89.     66.    ...   0.167  21.      0.   ]
 [  0.    137.     40.    ...   2.288  33.      1.   ]
 [  3.     78.     50.    ...   0.248  26.      1.   ]
 ...
 [  2.     88.     58.    ...   0.766  22.      0.   ]
 [ 10.    101.     76.    ...   0.171  63.      0.   ]
 [  5.    121.     72.    ...   0.245  30.      0.   ]]


In [26]:
trainingData = filtered_dataset[:274]
testData = filtered_dataset[274:]

In [27]:
siez = len(filtered_dataset)
print(siez)

392


In [28]:
data_size = len(testData)
print(data_size)

118


In [31]:
k = 16
TP = 0
FN = 0
FP = 0
TN = 0

for element in testData:
    label = new_computeKNN(trainingData, element, k)
    #print("True = ", element[-1], "Predict = ", predict(label))

    tp, fn, fp, tn = evaluate(int(element[-1]), new_predict(label))
    TP += tp
    FN += fn
    FP += fp
    TN += tn
#print(data_size)
print("TP =", TP) # True Positive: người bị bệnh được dự đoán là bị bệnh
print("FN =", FN) # False Negative: người bị bệnh được dự đoán là không có bệnh
print("FP =", FP) # False Positive: người không bị bệnh được dự đoán là bị bệnh
print("TN =", TN) # True Negative: người không bị bệnh được dự đoán là không bị bệnh

accuracy =round(float( (TP + TN)/data_size),2)
precision = float(TP/(TP+FP)) # tính tỉ lệ số người thực sự bị bệnh trong những người được dự đoán bị bệnh
recall = float(TP/(TP+FN)) #tính tỉ lệ dự đoán chính xác trong những người bị bệnh

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")

TP = 27
FN = 10
FP = 13
TN = 68
Accuracy: 81.00%
Precision: 67.50%
Recall: 72.97%


# Chuẩn hóa dữ liệu

In [32]:
# ko dc viet = dataset[0] do no se la 2 bien tham chieu den dataset[0]
max_item = np.array(filtered_dataset[0][:8])
min_item = np.array(filtered_dataset[0][:8])
for item in filtered_dataset:
    for i in range(8):
            if item[i] > max_item[i]:
                max_item[i] = item[i]
            if item[i] < min_item[i]:
                min_item[i] = item[i]
print(max_item)
print(min_item)
max_item = max_item.astype(float)
min_item = min_item.astype(float)

[ 17.   198.   110.    63.   846.    67.1    2.42  81.  ]
[ 0.    56.    24.     7.    14.    18.2    0.085 21.   ]


In [33]:
for item in filtered_dataset:
    for i in range(8):
        item_value = float(item[i])
        item[i] = float((item_value - min_item[i])/(max_item[i] - min_item[i]))

In [37]:
k = 15
TP = 0
FN = 0
FP = 0
TN = 0

for element in testData:
    label = new_computeKNN(trainingData, element, k)
    #print("True = ", element[-1], "Predict = ", predict(label))

    tp, fn, fp, tn = evaluate(int(element[-1]), new_predict(label))
    TP += tp
    FN += fn
    FP += fp
    TN += tn
#print(data_size)
print("TP =", TP) # True Positive: người bị bệnh được dự đoán là bị bệnh
print("FN =", FN) # False Negative: người bị bệnh được dự đoán là không có bệnh
print("FP =", FP) # False Positive: người không bị bệnh được dự đoán là bị bệnh
print("TN =", TN) # True Negative: người không bị bệnh được dự đoán là không bị bệnh

accuracy =round(float( (TP + TN)/data_size),2)
precision = float(TP/(TP+FP)) # tính tỉ lệ số người thực sự bị bệnh trong những người được dự đoán bị bệnh
recall = float(TP/(TP+FN)) #tính tỉ lệ dự đoán chính xác trong những người bị bệnh

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")

TP = 25
FN = 12
FP = 6
TN = 75
Accuracy: 85.00%
Precision: 80.65%
Recall: 67.57%


# CODE LẠI HÀM MAIN THÀNH 1 FUNCTION ĐI KO CỨ MỖI LẦN CHẠY PHẢI COPY 1 ĐỐNG LẠI DÀI